In [1]:
import dotenv
dotenv.load_dotenv()
import os
import weave
import wandb

In [2]:
wandb.login(key = os.getenv("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for zscalersre.wandb.io to your netrc file: /home/kilnaar/.netrc
wandb: Currently logged in as: cumbel (june-pov) to https://zscalersre.wandb.io. Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(entity = "june-pov", project = "gemma-tuner")

In [4]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType, AutoPeftModelForCausalLM
from trl import SFTTrainer
import torch

/home/kilnaar/anaconda3/envs/gemma-tune/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Base Model

In [5]:
model_id = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast = True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit = True,
    device_map = "auto",
    torch_dtype = torch.bfloat16
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.91it/s]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


## Adapter Training Setup

In [6]:
adapt_to_ds = load_dataset("Abirate/english_quotes", split = "train[:2%]")
target_adapter_name = "test-gemma-lora-adapter"

In [7]:
def format_fn(ex):
    return f"Quote: {ex['quote']}\nAuthor: {ex['author']}"

lora_cfg = LoraConfig(
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type = TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_cfg)

# --- Training -----------------------------------------------------
args = TrainingArguments(
    output_dir = target_adapter_name,
    per_device_train_batch_size = 4,
    num_train_epochs = 1,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 10,
    save_steps = 100,
    save_total_limit = 2,
)

trainer = SFTTrainer(
    model = model,
    args = args,
    train_dataset = adapt_to_ds,
    formatting_func = format_fn,
    processing_class = tokenizer,
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## 

## Training

In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/home/kilnaar/anaconda3/envs/gemma-tune/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/kilnaar/anaconda3/envs/gemma-tune/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=4, training_loss=1.6031160354614258, metrics={'train_runtime': 3.3561, 'train_samples_per_second': 14.898, 'train_steps_per_second': 1.192, 'total_flos': 52251385405440.0, 'train_loss': 1.6031160354614258})

In [9]:
model.save_pretrained(target_adapter_name)
tokenizer.save_pretrained(target_adapter_name)

('test-gemma-lora-adapter/tokenizer_config.json',
 'test-gemma-lora-adapter/special_tokens_map.json',
 'test-gemma-lora-adapter/tokenizer.model',
 'test-gemma-lora-adapter/added_tokens.json',
 'test-gemma-lora-adapter/tokenizer.json')

## Inference

In [10]:
ft_model = AutoPeftModelForCausalLM.from_pretrained(
    target_adapter_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
prompt = "Quote: We have nothing to fear but fear itself\nAuthor:"

print(tokenizer.decode(ft_model.generate(
        **tokenizer(prompt, return_tensors="pt").to(ft_model.device),
        max_new_tokens=16)[0], skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.03it/s]


Quote: We have nothing to fear but fear itself
Author: Franklin D. Roosevelt
Source: The New York Times, 193


In [11]:
prompt = "Quote: We choose to go to the moon\nAuthor:"

print(tokenizer.decode(ft_model.generate(
        **tokenizer(prompt, return_tensors="pt").to(ft_model.device),
        max_new_tokens=16)[0], skip_special_tokens=True))

Quote: We choose to go to the moon
Author: John F. Kennedy
Source: NASA

The moon is a symbol of the
